In [ ]:
# import standard Python packages
import os
import sys
import subprocess
from pathlib import Path

In [ ]:
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

In [ ]:
# import GRASS GIS python packages
import grass.script as gs
import grass.jupyter as gj

In [ ]:
# create a temporary folder where to place our GRASS project
import tempfile
tempdir = tempfile.TemporaryDirectory()

In [ ]:
gs.create_project(path=tempdir.name, name="nc_sentinel", epsg="32617", overwrite=True)

In [ ]:
# start GRASS in the recently created project
session = gj.init(Path(tempdir.name,"nc_sentinel"))